# Boston Neighborhoods Analysis

## Gather Neighborhood Data, Clean Data, and Visualize Data

### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

### Get data from excel file and load into a df

In [2]:
wb = load_workbook(filename='Boston Neighborhood Coordinates.xlsx', read_only=True)
ws = wb.active
col_names = [ws["A1"].value, ws["B1"].value, ws["C1"].value, ws["D1"].value]
header = np.array(col_names)

df = pd.DataFrame(ws.values, columns=header)
wb.close()

### Clean Data

In [3]:
df = df.drop(['Zip Code'], axis=1)
df = df.drop([0])
df.set_index('Neighborhood', inplace=True)

In [4]:
df

Latitude Longitude
Neighborhood                                 
Chinatown/Leather District  42.3515  -71.0602
West End                    42.3653  -71.0646
South End                   42.3378  -71.0706
Roxbury                     42.3218  -71.0884
Mission Hill                42.3319  -71.0988
Mattapan                    42.2771  -71.0914
South Boston                42.3365  -71.0349
East Boston                 42.3745   -71.032
Charlestown                 42.3795  -71.0646
Jamaica Plain               42.3097  -71.1151
Roslindale                  42.2832   -71.127
West Roxbury                42.2798  -71.1627
Allston                     42.3493    -71.13
Brighton                      42.35   -71.158
Hyde Park                   42.2528    -71.13
North Cambridge             42.3933  -71.1345
Brookline Village           42.3328  -71.1389
Chestnut Hill               42.3278  -71.1627
Downtown                    42.3573  -71.0562
North End                   42.3646  -71.0534
Beacon Hill                 42.3598  -71.0678
Back Bay                    42.3515  -71.0805
Dorchester                  42.2981  -71.0625
Harvard Square              42.3725  -71.1202
East Cambridge              42.3677   -71.081
Fenway/Kenmore              42.3441  -71.0957
Porter Square               42.3894  -71.1194
West Cambridge               42.377  -71.1359
Inman Square                42.3734  -71.0982
Central Square              42.3651  -71.1034
Davis Square                42.3963  -71.1222
Winter Hill                 42.3909  -71.0958
Assembly Square             42.3931  -71.0804
Seaport                     42.3492  -71.0415

## Create Map of Neighborhoods

In [5]:
import folium # map rendering library

In [6]:
# create map of Boston using latitude and longitude values
map_Boston = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)
    
    folium.Circle([lat, lng],
                    radius=1000
                   ).add_to(map_Boston)
    
map_Boston

## Get Venue Data, Clean Data, and Visualize Data

In [7]:
import requests
import json # library to handle JSON files

In [8]:
CLIENT_ID = 'GKVJ5KSFU1RD00P24CV5JT2XSARMWSONFYRMLT2EK2IJFEBW'
CLIENT_SECRET = 'SSJALJ4S5QXDNQBZMNUNFBE1XMVQZETMJWU1JDXE2VWK2PZL'
VERSION = '20180605' # Foursquare API version

### Function that extracts the category of the venue

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Function that gets venues within a given radius of the Neighborhood

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 1000
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        r = requests.get(url)
        print(r.status_code)
        results = r.json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
Boston_venues = getNearbyVenues(names=df.index, latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000)

Chinatown/Leather District
200
West End
200
South End
200
Roxbury
200
Mission Hill
200
Mattapan
200
South Boston
200
East Boston
200
Charlestown
200
Jamaica Plain
200
Roslindale
200
West Roxbury
200
Allston
200
Brighton
200
Hyde Park
200
North Cambridge 
200
Brookline Village
200
Chestnut Hill
200
Downtown
200
North End
200
Beacon Hill
200
Back Bay
200
Dorchester
200
Harvard Square 
200
East Cambridge
200
Fenway/Kenmore
200
Porter Square
200
West Cambridge
200
Inman Square
200
Central Square
200
Davis Square
200
Winter Hill
200
Assembly Square
200
Seaport
200


### Boston's Neighborhoods with Venues

In [12]:
print(Boston_venues.shape)
    
Boston_venues.nunique()

(2563, 7)


Neighborhood                34
Neighborhood Latitude       34
Neighborhood Longitude      31
Venue                     1908
Venue Latitude            2157
Venue Longitude           2158
Venue Category             291
dtype: int64

In [13]:
Boston_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Chinatown/Leather District                42.3515                -71.0602   
1  Chinatown/Leather District                42.3515                -71.0602   
2  Chinatown/Leather District                42.3515                -71.0602   
3  Chinatown/Leather District                42.3515                -71.0602   
4  Chinatown/Leather District                42.3515                -71.0602   

                             Venue  Venue Latitude  Venue Longitude  \
0                 Gracenote Coffee       42.351657       -71.058204   
1                        Shabu Zen       42.350922       -71.060942   
2  Hostelling International Boston       42.351084       -71.063398   
3                 Barry's Bootcamp       42.354010       -71.059776   
4              Tatte Bakery & Cafe       42.353111       -71.057117   

         Venue Category  
0           Coffee Shop  
1      Asian Restaurant  
2                Hostel  
3  Gym / Fitness Center  
4                Bakery

### Create Excel File with total venue counts per category

In [33]:
Venue_cat_count = Boston_venues.groupby('Venue Category').count()
Venue_cat_count.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_cat_count = pd.DataFrame(Venue_cat_count['Count'], Venue_cat_count.index)
Venue_cat_count.to_excel('Venue_Category_Counts.xlsx')
Venue_cat_count.sort_values('Count', ascending=False)

Count
Venue Category           
Pizza Place            94
Park                   92
Coffee Shop            86
Italian Restaurant     80
Bakery                 66
...                   ...
Eye Doctor              1
Field                   1
Food Service            1
Pilates Studio          1
ATM                     1

[291 rows x 1 columns]

### Create Excel File with total venue counts per neighborhood

In [34]:
Venue_per_hood = Boston_venues.groupby('Neighborhood').count()
Venue_per_hood.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_per_hood = pd.DataFrame(Venue_per_hood['Count'], Venue_per_hood.index)
Venue_per_hood.to_excel('Venue_Per_Hood.xlsx')
Venue_per_hood.sort_values('Count', ascending=False)

Count
Neighborhood                     
Allston                       100
North End                     100
Inman Square                  100
Assembly Square               100
Harvard Square                100
Fenway/Kenmore                100
East Cambridge                100
Downtown                      100
Seaport                       100
Davis Square                  100
Chinatown/Leather District    100
South End                     100
Central Square                100
West End                      100
Beacon Hill                   100
Back Bay                      100
Porter Square                 100
East Boston                    92
Brighton                       76
South Boston                   67
Winter Hill                    67
Chestnut Hill                  61
Jamaica Plain                  59
Mission Hill                   57
Roslindale                     52
Brookline Village              50
West Roxbury                   46
Dorchester                     41
Charlestown                    40
Hyde Park                      38
Mattapan                       34
North Cambridge                28
West Cambridge                 28
Roxbury                        27

### Create dummy variables to proportion venue categories for each neighborhood

In [16]:
# one hot encoding
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Boston_onehot['Neighborhood'] = Boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

In [17]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped

Neighborhood  Yoga Studio       ATM  Accessories Store  \
0                      Allston     0.010000  0.000000           0.000000   
1              Assembly Square     0.000000  0.000000           0.000000   
2                     Back Bay     0.010000  0.000000           0.000000   
3                  Beacon Hill     0.010000  0.000000           0.000000   
4                     Brighton     0.000000  0.000000           0.000000   
5            Brookline Village     0.000000  0.000000           0.000000   
6               Central Square     0.020000  0.000000           0.000000   
7                  Charlestown     0.025000  0.000000           0.000000   
8                Chestnut Hill     0.000000  0.000000           0.000000   
9   Chinatown/Leather District     0.000000  0.000000           0.000000   
10                Davis Square     0.020000  0.000000           0.000000   
11                  Dorchester     0.000000  0.000000           0.000000   
12                    Downtown     0.020000  0.000000           0.000000   
13                 East Boston     0.000000  0.000000           0.000000   
14              East Cambridge     0.000000  0.000000           0.000000   
15              Fenway/Kenmore     0.000000  0.000000           0.000000   
16             Harvard Square      0.010000  0.000000           0.000000   
17                   Hyde Park     0.026316  0.026316           0.000000   
18                Inman Square     0.000000  0.000000           0.000000   
19               Jamaica Plain     0.033898  0.000000           0.033898   
20                    Mattapan     0.000000  0.000000           0.000000   
21                Mission Hill     0.000000  0.000000           0.000000   
22            North Cambridge      0.000000  0.000000           0.000000   
23                   North End     0.020000  0.000000           0.000000   
24               Porter Square     0.020000  0.000000           0.000000   
25                  Roslindale     0.038462  0.000000           0.000000   
26                     Roxbury     0.000000  0.000000           0.000000   
27                     Seaport     0.000000  0.000000           0.000000   
28                South Boston     0.000000  0.000000           0.000000   
29                   South End     0.010000  0.000000           0.010000   
30              West Cambridge     0.000000  0.000000           0.000000   
31                    West End     0.020000  0.000000           0.000000   
32                West Roxbury     0.000000  0.000000           0.000000   
33                 Winter Hill     0.000000  0.000000           0.000000   

    Afghan Restaurant  African Restaurant   Airport  Airport Service  \
0                0.00            0.000000  0.000000          0.00000   
1                0.00            0.000000  0.000000          0.00000   
2                0.00            0.000000  0.000000          0.00000   
3                0.00            0.000000  0.000000          0.00000   
4                0.00            0.000000  0.000000          0.00000   
5                0.00            0.000000  0.000000          0.00000   
6                0.00            0.000000  0.000000          0.00000   
7                0.00            0.000000  0.000000          0.00000   
8                0.00            0.000000  0.000000          0.00000   
9                0.00            0.000000  0.000000          0.00000   
10               0.00            0.000000  0.000000          0.00000   
11               0.00            0.000000  0.000000          0.00000   
12               0.00            0.000000  0.000000          0.00000   
13               0.00            0.000000  0.021739          0.01087   
14               0.01            0.000000  0.000000          0.00000   
15               0.00            0.000000  0.000000          0.00000   
16               0.00            0.000000  0.000000          0.00000   
17               0.00            0.000000  0.000000          

### Get top 20 most occuring venues per neighborhood

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighborhood    1st Most Common Venue 2nd Most Common Venue  \
0                      Allston        Korean Restaurant    Chinese Restaurant   
1              Assembly Square           Clothing Store            Donut Shop   
2                     Back Bay           Clothing Store    Seafood Restaurant   
3                  Beacon Hill              Coffee Shop         Historic Site   
4                     Brighton              Pizza Place    Chinese Restaurant   
5            Brookline Village                     Park           Coffee Shop   
6               Central Square              Coffee Shop        Sandwich Place   
7                  Charlestown              Pizza Place             Gastropub   
8                Chestnut Hill           Clothing Store        Ice Cream Shop   
9   Chinatown/Leather District           Sandwich Place           Coffee Shop   
10                Davis Square       Mexican Restaurant        Breakfast Spot   
11                  Dorchester    Vietnamese Restaurant           Pizza Place   
12                    Downtown                     Park           Coffee Shop   
13                 East Boston      Rental Car Location    Mexican Restaurant   
14              East Cambridge                     Park                 Hotel   
15              Fenway/Kenmore      American Restaurant                 Hotel   
16             Harvard Square                      Café             Bookstore   
17                   Hyde Park              Pizza Place   American Restaurant   
18                Inman Square  New American Restaurant           Coffee Shop   
19               Jamaica Plain                   Bakery                  Park   
20                    Mattapan     Caribbean Restaurant           Pizza Place   
21                Mission Hill               Art Museum            Donut Shop   
22            North Cambridge                      Park                 Trail   
23                   North End       Italian Restaurant                  Park   
24               Porter Square                     Park                  Café   
25                  Roslindale              Pizza Place    Italian Restaurant   
26                     Roxbury                     Park     Convenience Store   
27                     Seaport       Seafood Restaurant                 Hotel   
28                South Boston              Coffee Shop                  Park   
29                   South End       Italian Restaurant              Wine Bar   
30              West Cambridge                Pet Store    Italian Restaurant   
31                    West End                    Hotel    Italian Restaurant   
32                West Roxbury              Pizza Place   American Restaurant   
33                 Winter Hill              Pizza Place            Donut Shop   

            3rd Most Common Venue      4th Most Common Venue  \
0                 Thai Restaurant         Mexican Restaurant   
1              Mexican Restaurant                Coffee Shop   
2                             Spa                       Park   
3                           Hotel         Italian Restaurant   
4                            Café          Convenience Store   
5                   Metro Station         Mexican Restaurant   
6   Vegetarian / Vegan Restaurant         Italian Restaurant   
7                  History Museum              National Park   
8              Seafood Restaurant         Salon / Barbershop   
9                          Bakery         Chinese Restaurant   
10                           Park          Indian Restaurant   
11                           Park                       Café   
12             Italian Restaurant                      Hotel   
13             Italian Restaurant  Latin American Restaurant   
14                    Coffee Shop        American Restaurant   
15                           Café                     Lounge   
16            American Restaurant                Pizza Place   
17             Chinese Restaur

## Create Relevant Functions

### df_style Function

In [20]:
def df_style(val):
    boolean = 0
    for item in match:
        if val == item:
            boolean = 1
        
    if boolean == 1:
        return 'font-weight: bold'
    else:
        return ''

### Main_Score Function

In [21]:
def main_score(df):

    score = []
    j=0
    for row in df.itertuples():
        i=0 
        for item in zip(row):
            if item[0] == match[0]:
                if i <= 2:
                    score.append(1)
                elif i <= 4:
                    score.append(.9)
                elif i <= 6:
                    score.append(.75)
                elif i <= 11:
                    score.append(.5)
                elif i <= 16:
                    score.append(.33)
                elif i <= 21:
                    score.append(.2)
                else:
                    score.append(0)
            i=i+1
        j=j+1
        if j > len(score):
            score.append(0)
            
    df['Main Score'] = score
    df.sort_values(by=['Main Score'], ascending=False, inplace=True)

### Match Function

In [22]:
def match_func(criterion):

    match = []
    for item in criterion:

        mask = neighborhoods_venues_sorted.applymap(lambda x: item in str(x))
        match_results = neighborhoods_venues_sorted[mask == True].dropna(how='all')

        for i in range(0,len(match_results.index)):
            results = []
            results = match_results.iloc[i,].unique()
            match.append(results[1])
            
    match = list(dict.fromkeys(match))
    return match

### Match_Score Function

In [23]:
def match_score(df):

    num_match = []
    score = []

    for row in df.itertuples():
        i=0
        for item in zip(row):
            for crit in match:
                if item[0] == crit:
                    i=i+1
        num_match.append(i)
        score.append(i/5)
    df['Number of Matches'] = num_match
    df['Match Score'] = score
    df.sort_values(by=['Number of Matches'], ascending=False, inplace=True)

### Total_Score Function

In [24]:
def total_score(df):
    tot_score = []
    rows = len(df.index)
    for i in range(0,rows):
        main_score = df.loc[df.index[i], 'Main Score']
        match_score = df.loc[df.index[i], 'Match Score']
        tot_score.append((main_score + match_score)/2)
    df['Total Score'] = tot_score
    df.sort_values(by=['Total Score'], ascending=False, inplace=True)

## Run Analysis of Neighborhoods based on Criterion

### Create df that includes Neighborhoods with PRIORITY criterion

In [25]:
criterion = ['Italian', 'Pizza', 'Ice Cream', 'Gym', 'Pub'] #First term is priority venue
match = []

mask = neighborhoods_venues_sorted.applymap(lambda x: criterion[0] in str(x))
search_priority = neighborhoods_venues_sorted[mask == True].dropna(how='all')

for i in range(0,len(search_priority.index)):
    results = []
    results = search_priority.iloc[i,].unique()
    match.insert(0, results[1])
match = list(dict.fromkeys(match))

search_priority = neighborhoods_venues_sorted.iloc[search_priority.index]
match = match_func(criterion)
main_score(search_priority)
match_score(search_priority)
total_score(search_priority)
    
i=0
for items in zip(search_priority['Neighborhood']):
    i=i+1
print(f"{i} Results Found!")

search_priority.style.applymap(df_style)

C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

19 Results Found!


### Create df that includes Neighborhoods with ALL criterion

In [26]:
search_perf = search_priority.loc[search_priority['Number of Matches'] == 5]

rows = len(search_perf.index)
print(f"{rows} Results Found!")

if rows == 0:
    print("Criterion too specific")

search_perf.style.applymap(df_style)

1 Results Found!


### Final Score Result for all Neighborhoods

In [54]:
criterion = ['Italian', 'Pizza', 'Ice Cream', 'Gym', 'Pub'] #First term is priority venue
match = match_func(criterion)
main_score(neighborhoods_venues_sorted)
match_score(neighborhoods_venues_sorted)
total_score(neighborhoods_venues_sorted)
neighborhoods_venues_sorted.style.applymap(df_style)

## Map all Italian Restaurants in Boston

In [28]:
Boston_Italian = Boston_venues.loc[Boston_venues['Venue Category'] == 'Italian Restaurant']
Boston_Italian.drop(['Neighborhood Latitude', 'Neighborhood Longitude'], axis=1, inplace=True)
fixed_columns = [Boston_Italian.columns[1]] + list(Boston_Italian.columns[2:4]) + [Boston_Italian.columns[0]] + [Boston_Italian.columns[-1]]
Boston_Italian = Boston_Italian[fixed_columns]

C:\Users\jhenn\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
Boston_Italian

Venue  Venue Latitude  Venue Longitude  \
84        Maggiano's Little Italy       42.350966       -71.068695   
111               Osteria Rustico       42.364397       -71.059870   
121   Scampo at The Liberty Hotel       42.361937       -71.070513   
124                        Grotto       42.359915       -71.062807   
177                          Aria       42.363286       -71.055002   
...                           ...             ...              ...   
2498                    Sportello       42.350519       -71.048687   
2522            Sapóre Ristorante       42.348126       -71.039863   
2530               Buco Trattoria       42.349923       -71.049393   
2536               Tuscan Kitchen       42.352829       -71.047298   
2547            Strega Waterfront       42.353062       -71.045162   

                    Neighborhood      Venue Category  
84    Chinatown/Leather District  Italian Restaurant  
111                     West End  Italian Restaurant  
121                     West End  Italian Restaurant  
124                     West End  Italian Restaurant  
177                     West End  Italian Restaurant  
...                          ...                 ...  
2498                     Seaport  Italian Restaurant  
2522                     Seaport  Italian Restaurant  
2530                     Seaport  Italian Restaurant  
2536                     Seaport  Italian Restaurant  
2547                     Seaport  Italian Restaurant  

[80 rows x 5 columns]

In [46]:
# create map of Boston using latitude and longitude values
map_Italian = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add Restaurant markers to map
for lat, lng, venue, neighborhood in zip(Boston_Italian['Venue Latitude'], Boston_Italian['Venue Longitude'], Boston_Italian['Venue'], Boston_Italian['Neighborhood']):
    label = f'{venue}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Italian)
    
# add Neighborhood markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    
    folium.Circle(
        [lat, lng],
        radius=1000,
        popup=label).add_to(map_Italian)
    
map_Italian

## Map Top Neighborhood

In [69]:
Top_Hood = Boston_venues.loc[Boston_venues['Neighborhood'] == neighborhoods_venues_sorted.iloc[0]['Neighborhood']]
Top_Hood

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
200    South End                42.3378                -71.0706   
201    South End                42.3378                -71.0706   
202    South End                42.3378                -71.0706   
203    South End                42.3378                -71.0706   
204    South End                42.3378                -71.0706   
..           ...                    ...                     ...   
295    South End                42.3378                -71.0706   
296    South End                42.3378                -71.0706   
297    South End                42.3378                -71.0706   
298    South End                42.3378                -71.0706   
299    South End                42.3378                -71.0706   

                      Venue  Venue Latitude  Venue Longitude  \
200                  Stella       42.339736       -71.072574   
201             anoush’ella       42.338869       -71.073843   
202     Flour Bakery & Cafe       42.338447       -71.074299   
203             The Gallows       42.341262       -71.070135   
204  Boston Flower Exchange       42.337819       -71.066743   
..                      ...             ...              ...   
295             Lion's Tail       42.344888       -71.063497   
296     BURN Fitness Studio       42.341588       -71.080917   
297           House of Siam       42.343157       -71.073138   
298             Bar Mezzana       42.344562       -71.063560   
299       Fuji at Ink Block       42.345096       -71.063528   

               Venue Category  
200        Italian Restaurant  
201  Mediterranean Restaurant  
202                    Bakery  
203                 Gastropub  
204               Flower Shop  
..                        ...  
295                 Speakeasy  
296                       Gym  
297           Thai Restaurant  
298        Italian Restaurant  
299       Japanese Restaurant  

[100 rows x 7 columns]

In [70]:
# create map of Boston using latitude and longitude values
latitude = Top_Hood.iloc[0]['Neighborhood Latitude']
longitude = Top_Hood.iloc[0]['Neighborhood Longitude']
map_Top_Hood = folium.Map(location=[latitude, longitude], zoom_start=15)

# add Restaurant markers to map
for lat, lng, venue in zip(Top_Hood['Venue Latitude'], Top_Hood['Venue Longitude'], Top_Hood['Venue']):
    label = f'{venue}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Top_Hood)
    
# add Neighborhood markers to map
label = 'South End'
label = folium.Popup(label, parse_html=True)
    
folium.Circle(
    [latitude, longitude],
    radius=1000,
    popup=label).add_to(map_Top_Hood)

map_Top_Hood